In [1]:
# set the environment variables needed for openai package to know to reach out to azure
import os

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_BASE"] = "https://<your-endpoint.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "your AzureOpenAI key"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"

from dotenv import load_dotenv
load_dotenv("../.env", override=True)

True

In [44]:
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings

llm = AzureChatOpenAI(deployment_name="gpt-35-turbo", temperature=0.5)
embeddings = OpenAIEmbeddings(deployment="text-embedding-ada-002", chunk_size=1)

In [3]:
from langchain.vectorstores.redis import Redis

rds = Redis.from_existing_index(
    embeddings, redis_url="redis://localhost:6379", index_name="link"
)
retriever = rds.as_retriever(search_type="similarity_limit", score_threshold=0.8)

In [4]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=llm, chain_type="map_reduce", retriever=retriever)

In [34]:
from langchain.agents import Tool

tools = [
    Tool(
        name="Contract QA System",
        func=qa.run,
        description="useful for when you need to answer questions about contracts.Input needs to be in Chinese",
    )
]

In [35]:
from langchain.agents import initialize_agent
from langchain.agents import AgentType

In [46]:
agent_chain = initialize_agent(
    tools, llm, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

agent_chain.run("在什么情况下可以单方面解除合同？")



> Entering new  chain...
Thought: This question is asking about the circumstances under which one party can unilaterally terminate a contract. I will use the Contract QA System tool to find the answer.

Action:
```
{
  "action": "Contract QA System",
  "action_input": "在什么情况下可以单方面解除合同？"
}
```


Observation: 根据文本，可以单方面解除合同的情况如下：

甲方可以单方面解除合同的情况：
- 在承担国家重大科研项目期间；
- 掌握重大科技成果关键技术和资料，未脱离保密期；
- 被审查期间或因经济问题未作结案处理之前；
- 合同订立时所依据的客观情况发生重大变化，致使合同无法履行，经甲乙双方协商不能就变更合同达成协议的；
- 甲方提出解除本合同，乙方同意解除的；
- 乙方患病或者非因工负伤，医疗期满后，不能从事原工作也不能从事由甲方安排的其他工作，甲方单方面解除本合同的；
- 乙方年度考核或者聘期考核不合格，又不同意甲方调整其工作岗位的；
- 乙方公派出国或因私出境逾期不归；
- 违反工作规定或者操作规程，发生责任事故，或者失职、渎职，造成严重后果的；
- 严重扰乱工作秩序，致使甲方、其他单位工作不能正常进行的；
- 被判处拘役、有期徒刑缓刑以及有期徒刑以上刑罚收监执行，或者被劳动教养的；
- 甲方被依法注销、撤销或者解散的。

乙方可以单方面解除合同的情况：
- 在试用期内的；
- 考入普通高等院校的；
- 被录用或者选调为公务员的；
- 依法服兵役的；
- 如果乙方提出解除合同未能与甲方协商一致，乙方应当坚持正常工作，继续履行本合同；6个月后再次提出解除本合同仍未能与甲方协商一致的，即可单方面解除本合同。

此外，乙方患病或者非因工负伤，医疗期满后，不能从事原工作也不能从事由甲方安排的其他工作的情况下，甲方可以单方面解除合同，但是应当提前30日以书面形式通知乙方。乙方按照国家有关规定退休或退职、死亡或者被人民法院宣告死亡的情况下，合同也会自动终止。
Thought

'根据合同法，甲方和乙方可以在特定情况下单方面解除合同。甲方和乙方都有各自的解除合同情况。'

In [ ]:
agent_chain = initialize_agent(
    tools, llm, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

agent_chain("在什么情况下可以单方面解除合同？")

In [49]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history",return_messages=True)
agent_chain = initialize_agent(tools, llm, agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory)

In [51]:
response = agent_chain(
    {
        "input": "在什么情况下可以解除合同？"
    }
)



> Entering new  chain...
```json
{
    "action": "Contract QA System",
    "action_input": "在什么情况下可以解除合同？"
}
```
Observation: 根据文本，以下情况可以解除合同：

1. 在承担国家重大科研项目期间；
2. 掌握重大科技成果关键技术和资料，未脱离保密期；
3. 被审查期间或因经济问题未作结案处理之前；
4. 合同订立时所依据的客观情况发生重大变化，致使合同无法履行，经甲乙双方协商不能就变更合同达成协议的；
5. 甲方提出解除本合同，乙方同意解除的；
6. 乙方患病或者非因工负伤，医疗期满后，不能从事原工作也不能从事由甲方安排的其他工作，甲方单方面解除本合同的；
7. 乙方年度考核或者聘期考核不合格，又不同意甲方调整其工作岗位的；
8. 乙方有下列情形之一的，甲方可以随时单方面解除本合同：
   1. 在试用期内被证明不符合本岗位要求；
   2. 连续旷工超过 10 个工作日或者 1 年内累计旷工超过 20 个工作日的；
   3. 乙方公派出国或因私出境逾期不归；
   4. 违反工作规定或者操作规程，发生责任事故，或者失职、渎职，造成严重后果的；
   5. 严重扰乱工作秩序，致使甲方、其他单位工作不能正常进行的；
   6. 被判处拘役、有期徒刑缓刑以及有期徒刑以上刑罚收监执行，或者被劳动教养的。
9. 乙方有下列情形之一，甲方可以单方面解除本合同，但是应当提前30日以书面形式通知乙方：
   1. 乙方患病或者非因工负伤，医疗期满后，不能从事原工作也不能从事由甲方安排的其他工作的；
10. 甲方、乙方双方经协商一致，可以解除本合同。
Thought:```json
{
    "action": "Final Answer",
    "action_input": "根据文本，以下情况可以解除合同：\n\n1. 在承担国家重大科研项目期间；\n2. 掌握重大科技成果关键技术和资料，未脱离保密期；\n3. 被审查期间或因经济问题未作结案处理之前；\n4. 合同订立时所依据的客观情况发生重大变化，致使合同无法履行，经甲乙双方协商不能就变更合同达成协议的；\n5. 甲方提出解除本合同，乙方同意解除的；\n6. 乙方患病或者非因工负伤，医疗

In [53]:
agent_chain.memory.buffer

[HumanMessage(content='在什么情况下可以单方面解除合同？', additional_kwargs={}, example=False),
 AIMessage(content='根据文本，以下情况可以单方面解除合同：\n\n- 在承担国家重大科研项目期间；\n- 掌握重大科技成果关键技术和资料，未脱离保密期；\n- 被审查期间或因经济问题未作结案处理之前；\n- 合同订立时所依据的客观情况发生重大变化，致使合同无法履行，经甲乙双方协商不能就变更合同达成协议的；\n- 甲方提出解除本合同，乙方同意解除的；\n- 乙方患病或者非因工负伤，医疗期满后，不能从事原工作也不能从事由甲方安排的其他工作，甲方单方面解除本合同的；\n- 乙方年度考核或者聘期考核不合格，又不同意甲方调整其工作岗位的；\n- 乙方有下列情形之一的，甲方可以随时单方面解除本合同： \n  1．在试用期内被证明不符合本岗位要求； \n  2．连续旷工超过 10 个工作日或者 1 年内累计旷工超过 20 个工作日的； \n  3．乙方公派出国或因私出境逾期不归； \n  4．违反工作规定或者操作规程，发生责任事故，或者失职、渎职，造成严重后果的； \n  5．严重扰乱工作秩序，致使甲方、其他单位工作不能正常进行的； \n  6．被判处拘役、有期徒刑缓刑以及有期徒刑以上刑罚收监执行，或者被劳动教养的。 \n- 乙方有下列情形之一，甲方可以单方面解除本合同，但是应当提前30日以书面形式通知乙方： \n  1．乙方患病或者非因工负伤，医疗期满后，不能从事原工作也不能从事由甲方安排的其他工作的；\n- 乙方可以在以下情况下随时单方面解除合同：在试用期内的；考入普通高等院校的；被录用或者选调为公务员的；依法服兵役的。除此之外，如果乙方提出解除合同未能与甲方协商一致，乙方应当坚持正常工作，继续履行合同；6个月后再次提出解除合同仍未能与甲方协商一致的，即可单方面解除合同。\n- 有下列情形之一的，本合同终止： \n  1．本合同期限届满； \n  2．甲、乙双方约定的合同终止条件出现； \n  3．乙方按照国家有关规定退休或退职的； \n  4．乙方死亡或者被人民法院宣告死亡的； \n  5．甲方被依法注销、撤销或者解散的。', additional_kwargs={}, example=False

In [ ]:
from langchain.chains import RetrievalQAWithSourcesChain